In [ ]:
import sys
sys.path.append("/home/qrr2/project/")
sys.path.append("/home/qrr2/project/QOGS/")
import numpy as np
from scipy.ndimage import gaussian_filter
import qutip as qt
import matplotlib
import matplotlib.pyplot as plt
import csv
from pathlib import Path
from qutip import fidelity, basis, tensor, mesolve, fock, qeye, destroy, sesolve
import qutip.control.pulseoptim as cpo
import pandas as pd
import logging

# Initialize the logger correctly
logger = logging.getLogger()
log_level = logging.INFO  
logger.setLevel(log_level)

# Set up console logging
ch = logging.StreamHandler()
ch.setLevel(log_level)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

matplotlib.get_backend()

### Definitions

In [ ]:
dim_qubit = 2 # Qubit dimension
dim_resonator = 6 # Resontor dimension
Id = tensor(qeye(dim_qubit), qeye(dim_resonator))
q = tensor(destroy(dim_qubit), qeye(dim_resonator)) # Qubit
r = tensor(qeye(dim_qubit), destroy(dim_resonator)) # Phonon mode
qd = q.dag()                  
rd = r.dag()  

target_fock = 3 # Target fock integer
g = 0.28 * np.pi * 2 # In MHz
H_static = g * (rd * q + r * qd)
Hcs = [q + qd, 1j*(q - qd)]
n_cs = len(Hcs)

init_state = tensor(basis(dim_qubit, 0), fock(dim_resonator, 0))
target_state = tensor(basis(dim_qubit, 0), fock(dim_resonator, target_fock))

### Optimization

In [ ]:
# Define optimization parameters
n_ts = 750 # Number of time slices
evo_time = 3 # Pulse length in (*@\textcolor{codegreen}{$\mu$}@*)s
fid_err_targ = 1e-20 # Target infedility
max_iter = 10000 # Maximum number of iterations
max_wall_time = 2000 # Maximum real-world time it will run in s
p_type = 'SINE' # Type of pulse basis   
f_ext = 'pulse_sequence.csv' 
method_params = {'xtol': 1e-6} # Tolerance for convergence

result = cpo.opt_pulse_crab_unitary(H_static, Hcs, init_state, 
         target_state, n_ts, evo_time, fid_err_targ=fid_err_targ,
         max_iter=max_iter, max_wall_time=max_wall_time,
         method_params=method_params,
         guess_pulse_type=p_type, guess_pulse_action="modulate",
         out_file_ext=f_ext,
         log_level=log_level, gen_stats=True)

### Pulse

In [ ]:
# Extract I and Q components
I = result.final_amps[:, 0]
Q = result.final_amps[:, 1]

# Convert lists to numpy arrays
I = np.array(I, dtype=float)
Q = np.array(Q, dtype=float)
time = np.arange(evo_time, step=(evo_time/n_ts), dtype=float)

# Plot I and Q components vs time
fig, ax = plt.subplots(figsize=(12, 4))
ax.set_facecolor("whitesmoke")
plt.plot(time*1000, I, label="Real", color="#440154FF")
plt.plot(time*1000, Q, label="Imaginary", color="#2C728EFF")
plt.xlabel("Time (ns)")
plt.ylabel("Amplitude (MHz)")
plt.title("Control Pulses: I and Q vs Time")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

### Calculate Fidelity

In [ ]:
# Construct the time-dependent Hamiltonian
H_tot = [H_static] + [[Hcs[0], I], [Hcs[1], Q]]

# Simulate the evolution of the system
result = qt.sesolve(H_tot, init_state, time, [])

# Calculate the fidelity
final_state = result.states[-1]
fidelity = np.abs(final_state.overlap(target_state)) ** 2

# Print the fidelity
print(f"Fidelity: {fidelity}")

### Plot Density Matrix

In [ ]:
final_state = result.states[-1]
# Compute the density matrix of the final state
density_matrix = final_state * final_state.dag()

# Convert the density matrix to a NumPy array
density_matrix_np = density_matrix.full()

# Visualize the density matrix
fig, ax = qt.hinton(density_matrix)
plt.title('Density Matrix')
plt.show()

### Plot Average Population Evolution

In [ ]:
# Number operators for the resonator
n_r = qt.tensor(qt.qeye(dim_qubit), qt.num(dim_resonator))

# List of initial states for which to plot the population evolution
initial_states_to_plot = [tensor(basis(dim_qubit, 0), fock(dim_resonator, 0))]

for init_state in initial_states_to_plot:

    # Simulate the evolution
    result = qt.mesolve(H_tot, init_state, time, [], e_ops=[n_r])

    # Plotting
    plt.figure(figsize=(10, 6))

    # Plot the expectation values for each cavity
    plt.plot(time, result.expect[0], lw=2, label='Resonator')

    plt.xlabel('Time (ns)', fontsize=12)
    plt.ylabel('Average Population', fontsize=12)
    plt.title(f'Average Population in HBAR Over Time', fontsize=14)
    plt.legend(fontsize=10)
    plt.tight_layout()
    plt.show()

### Save CSV

In [ ]:
# CSV file ready for Gandalf
output_file = "CRAB_control.csv"
expanded_rows = []
for i, q in zip(I, Q):
    for _ in range(4):
        expanded_rows.append([i, q])
expanded_df = pd.DataFrame(expanded_rows)
expanded_df.insert(0, "Time", range(len(expanded_df)))
expanded_df.to_csv(output_file, index=False, header=False)
print(f"CSV has been saved to {output_file}")